In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-24"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14363,2024-11-24,Eua Nba,00:30,Los Angeles Lakers,Denver Nuggets,1.57,2.52,234.5,1.84,2.00,-4.5,1.93,1.93,KEn2icO8,0.636943,0.396825,0.543478,0.500000,0.033768,0.6,1.0,NaN,NaN,158.868,14.146714,0.089047,248.278,80.156690,0.322851,158.868,228.932,174.64,294.02,0.0,0.0,0.0,0.0,0.328485,0.058926,0.000000,0.05,0.010,57.000000,0.628658,0.8,0.171342,1.69,0.338,4.497041,0.522699,0.6,0.077301
14364,2024-11-24,Eua Nba,17:30,Boston Celtics,Minnesota Timberwolves,1.32,3.62,224.5,1.88,1.96,-8.5,1.98,1.88,hSpAkygL,0.757576,0.276243,0.531915,0.510204,0.033819,0.4,0.6,NaN,NaN,144.448,33.491651,0.231860,132.408,10.901742,0.082334,145.460,145.490,183.60,139.65,0.0,0.0,0.0,0.0,0.658440,0.029463,0.036638,-1.12,-0.224,-1.428571,0.707415,0.6,-0.107415,-3.70,-0.740,-3.540541,0.644194,0.3,-0.344194
14365,2024-11-24,Eua Nba,19:00,Indiana Pacers,Washington Wizards,1.10,5.43,241.5,1.83,1.83,-12.5,2.01,1.68,CCRtpFGr,0.909091,0.184162,0.546448,0.546448,0.093253,1.0,0.6,NaN,NaN,240.216,95.350641,0.396937,527.552,208.587774,0.395388,247.972,477.850,286.44,587.60,0.0,1.0,0.0,0.0,0.937756,0.000000,0.126474,3.17,0.634,0.157729,0.583531,0.6,0.016469,-1.27,-0.254,-17.440945,0.324906,0.3,-0.024906
14366,2024-11-24,Europa Campeonato Mundial,07:00,Azerbaijan,Suiça,6.05,1.11,144.5,1.87,1.87,11.5,2.03,1.70,AVs3N2D3,0.165289,0.900901,0.534759,0.534759,0.066190,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,191.40,0.00,0.0,0.0,0.0,0.0,0.975728,0.000000,0.125118,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
14367,2024-11-24,Europa Campeonato Mundial,11:00,Armênia,Áustria,4.26,1.21,165.5,1.87,1.86,8.5,2.01,1.72,0vmNEVQ0,0.234742,0.826446,0.534759,0.537634,0.061188,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,109.89,0.0,0.0,0.0,0.0,0.788547,0.003791,0.109952,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14485,2024-11-24,Eua Ncaa,19:00,UC Riverside,Alcorn State,1.06,11.00,140.5,1.91,1.91,-18.5,2.05,1.71,hEylCB1m,0.943396,0.090909,0.523560,0.523560,0.034305,0.8,0.2,NaN,NaN,539.064,650.452319,1.206633,824.140,786.674813,0.954540,133.342,817.364,146.08,1680.00,0.0,1.0,0.0,0.0,1.165612,0.000000,0.127881,-0.07,-0.014,-4.285714,0.585026,0.6,0.014974,-1.84,-0.368,-27.173913,0.374502,0.5,0.125498
14486,2024-11-24,Eua Ncaa,21:00,Montana,Denver,1.19,4.91,149.5,1.80,1.95,-11.5,1.95,1.77,6cD6aYHs,0.840336,0.203666,0.555556,0.512821,0.044002,0.6,0.2,NaN,NaN,207.614,193.064512,0.929920,406.210,452.407091,1.113727,111.174,421.112,106.60,1170.00,0.0,1.0,0.0,0.0,0.862438,0.056569,0.068430,0.54,0.108,1.759259,0.749523,0.7,-0.049523,-3.05,-0.610,-6.409836,0.363724,0.2,-0.163724
14487,2024-11-24,Eua Ncaa,21:00,USC,Grambling St.,1.05,11.50,139.5,1.91,1.91,-19.5,1.95,1.80,h6DoxA14,0.952381,0.086957,0.523560,0.523560,0.039337,0.6,0.4,NaN,NaN,97.454,25.063798,0.257186,917.624,875.608898,0.954213,126.870,933.864,86.10,562.60,0.0,1.0,0.0,0.0,1.177572,0.000000,0.056569,2.86,0.572,0.087413,0.723211,0.8,0.076789,-2.58,-0.516,-20.348837,0.348666,0.6,0.251334
14488,2024-11-24,Grécia A1 Feminina,10:00,Amyntas F,PAOK F,1.07,7.11,137.5,1.88,1.82,-14.5,1.92,1.75,8v5JsM6d,0.934579,0.140647,0.531915,0.549451,0.075226,0.0,0.0,NaN,NaN,NaN,NaN,NaN,311.596,276.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Lakers,Denver Nuggets,234.5,1.84,0.9838,Over
1,17:30,Eua Nba,Boston Celtics,Minnesota Timberwolves,224.5,1.88,0.9738,Over
2,20:00,Eua Nba,Miami Heat,Dallas Mavericks,221.5,1.84,0.9722,Over
3,21:30,Eua Nba,Cleveland Cavaliers,Toronto Raptors,235.5,1.81,0.8883,Over
4,23:00,Eua Nba,Sacramento Kings,Brooklyn Nets,220.5,1.81,0.9854,Over
5,17:00,Eua Ncaa,Akron,Alabama State,147.5,1.91,0.8470,Over
6,17:00,Eua Ncaa,Northwestern St.,North Alabama,145.5,1.91,0.8432,Over
